In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [4]:
df =pd.read_csv("customer_prod.csv")
# df.drop(columns=["customer_id","predictions","ct_Last_training"], inplace=True)
# df.drop_duplicates(inplace=True)

In [6]:
pd.DataFrame(columns=df.columns)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,predictions,churn


In [3]:
from build_pipeline import test_transforms
df_tf =test_transforms(df)

In [11]:
X_train, X_test, y_train, y_test= train_test_split( df.drop(columns=["churn"]), df["churn"], test_size=0.3, random_state=42)

In [14]:
from train_pipeline import train_pipeline


In [15]:
model = train_pipeline(X_train, y_train)

In [16]:
model

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",1.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept:

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [18]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1 Score :", f1_score(y_test, y_pred))
print("ROC-AUC  :", roc_auc_score(y_test, y_proba))
print("-" * 40)

Accuracy : 0.8037825059101655
Precision: 0.9491525423728814
Recall   : 0.7593220338983051
F1 Score : 0.8436911487758946
ROC-AUC  : 0.8840042372881356
----------------------------------------


### Airflow Tasks

#### training and finetuning

In [1]:
import os
import mlflow
import mlflow.sklearn
from mlflow import MlflowClient
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from build_pipeline import build_pipeline
import logging


### Train and finetuning task

In [2]:
import pandas as pd

def fetch_data_from_rds(table_name):

    RDS_HOST = "mlprojectdb.cx4k04y48cjc.eu-north-1.rds.amazonaws.com"
    RDS_PORT = 5432
    RDS_USER = "postgres"
    RDS_PASSWORD = "mlpipeline"
    RDS_DB_NAME = "telco_churn"
    TABLE_NAME = table_name
    
    # Création de l'URL de connexion avec psycopg2 explicite
    connection_url = f"postgresql+psycopg2://{RDS_USER}:{RDS_PASSWORD}@{RDS_HOST}:{RDS_PORT}/{RDS_DB_NAME}"

    try:
        print("Connexion à RDS en cours...")
        # Requête SQL simple
        query = f"SELECT * FROM {TABLE_NAME}"
        
        # Passer directement l'URL de connexion à pandas (sans créer d'engine)
        ref_df = pd.read_sql_query(query, connection_url)
        
        print(f"Extraction réussie : {len(ref_df)} lignes de réf")
        return ref_df

    except Exception as e:
        print(f"Erreur lors de la connexion à RDS : {e}")
        return None


In [3]:
import pandas as pd
from sqlalchemy import create_engine

def insert_dataframe_into_rds(df,tale_name):
    RDS_HOST = "mlprojectdb.cx4k04y48cjc.eu-north-1.rds.amazonaws.com"
    RDS_PORT = 5432
    RDS_USER = "postgres"
    RDS_PASSWORD = "mlpipeline"
    RDS_DB_NAME = "telco_churn"
    TABLE_NAME = tale_name #model_eval #prod_customers

    # SQLAlchemy connection URL
    connection_url = (
        f"postgresql+psycopg2://{RDS_USER}:{RDS_PASSWORD}"
        f"@{RDS_HOST}:{RDS_PORT}/{RDS_DB_NAME}"
    )

    try:
        print("Connexion à RDS...")
        engine = create_engine(connection_url)

        print("Insertion du DataFrame en cours...")
        df.to_sql(
            name=TABLE_NAME,
            con=engine,
            if_exists="append",   # 'replace' if you want to drop & recreate
            index=False,
            method="multi"        # faster bulk insert
        )

        print(f"Insertion réussie : {len(df)} lignes insérées dans {TABLE_NAME}")

    except Exception as e:
        print(f"Erreur lors de l'insertion dans RDS : {e}")


In [24]:
# fetch the data from RDS
customers = fetch_data_from_rds("prod_customers")
customers.drop_duplicates(inplace=True)

customers.drop(columns=["customer_id","predictions"], inplace=True)

#select the new and old customers
new_customers = customers[customers['ct_Last_training'] == False]
old_customers = customers[customers['ct_Last_training'] == True]

new_customers.drop(columns=["ct_Last_training"], inplace=True)
old_customers.drop(columns=["ct_Last_training"], inplace=True)

#shuffle the new customers
new_customers = new_customers.sample(frac=1, random_state=42)

#split the new customers into two parts: 80% for training and 20% for testing
eval_model_customers = new_customers[:int(new_customers.shape[0]*.2)]
new_customers = new_customers[int(new_customers.shape[0]*.2):]
#Store the eval_model_customers into RDS table model_eval
new_customers.to_parquet("new_customers.parquet", index=False)
# insert_dataframe_into_rds(eval_model_customers,"model_eval")



train_customers = pd.concat([old_customers, new_customers])


Connexion à RDS en cours...
Extraction réussie : 5834 lignes de réf


C:\Users\Admin\AppData\Local\Temp\ipykernel_9076\3488247465.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_customers.drop(columns=["ct_Last_training"], inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9076\3488247465.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_customers.drop(columns=["ct_Last_training"], inplace=True)


In [25]:
# from preprocess import apply_static_transforms
# # train_customers = apply_static_transforms(train_customers)

In [26]:
import logging
import mlflow
from sklearn.model_selection import GridSearchCV

def task_tune_and_train(train_customers):
    # Initialize variables to None to avoid UnboundLocalError
    reg_result = None
    grid = None
    
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("Churn_Retraining_Experiment")
    
    mlflow.sklearn.autolog(
        log_input_examples=True,
        log_model_signatures=True
    )

    try:
        with mlflow.start_run(run_name="grid_search_tuning") as run:
            # 1. Prepare Data
            # Check for NaN values here! If build_pipeline handles them, you're fine.
            X = train_customers.drop(columns=["churn"])
            y = train_customers["churn"]

            # 2. Build and Fit
            pipeline = build_pipeline()
            
            param_grid = {
                'model__C': [0.01, 0.1, 1, 10],
                'model__penalty': ['l1', 'l2'],
                'model__solver': ['liblinear', 'saga']
            }

            grid = GridSearchCV(
                estimator=pipeline, 
                param_grid=param_grid, 
                scoring='recall', 
                cv=5, 
                n_jobs=-1,
                refit=True 
            )
            
            # This is where your sample mismatch [4616, 4636] is occurring
            grid.fit(X, y)

            # 3. Register Model
            model_uri = f"runs:/{run.info.run_id}/model"
            reg_result = mlflow.register_model(
                model_uri=model_uri,
                name="churn_model"
            )
            
            return {
                "status": "success",
                "model_version": reg_result.version,
                "best_recall": grid.best_score_,
                "run_id": run.info.run_id
            }

    except Exception as e:
        logging.error(f"❌ Training failed: {e}")
        # Return a failure state instead of letting 'finally' crash the script
        return {
            "status": "failed",
            "error": str(e),
            "model_version": None,
            "best_recall": 0
        }

In [27]:
train_customers.dtypes

gender                object
senior_citizen          bool
partner                 bool
dependents              bool
tenure                 int64
phone_service           bool
multiple_lines        object
internet_service      object
online_security       object
online_backup         object
device_protection     object
tech_support          object
streaming_tv          object
streaming_movies      object
contract              object
paperless_billing       bool
payment_method        object
monthly_charges      float64
total_charges        float64
churn                   bool
dtype: object

In [28]:
results = task_tune_and_train(train_customers)

2026/01/27 09:27:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Admin\anaconda3\envs\ml1\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
c:\Users\Admin\anaconda3\envs\ml1\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 an

2026/01/27 09:28:57 INFO mlflow.sklearn.utils: Logging the 5 best runs, 11 runs will be omitted.
2026/01/27 09:28:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Admin\anaconda3\envs\ml1\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
Registered model 'churn_model' already exists. Creat

🏃 View run grid_search_tuning at: http://localhost:5000/#/experiments/1/runs/394cc5cfe7e84c30ad1169041b1c194a
🧪 View experiment at: http://localhost:5000/#/experiments/1


### Model Evaluation 

In [30]:
results

{'status': 'success',
 'model_version': '4',
 'best_recall': np.float64(0.7584090909090909),
 'run_id': '394cc5cfe7e84c30ad1169041b1c194a'}

In [31]:
eval_model_customers = pd.read_parquet("new_customers.parquet")
X_test = eval_model_customers.drop(["churn"],axis=1)
y_test = eval_model_customers["churn"]

# --- 4. LOAD MODELS ---
# Load the new Challenger
challenger_uri = f"models:/churn_model/{results['model_version']}"
challenger_model = mlflow.sklearn.load_model(challenger_uri)

In [32]:
train_customers.dtypes

gender                object
senior_citizen          bool
partner                 bool
dependents              bool
tenure                 int64
phone_service           bool
multiple_lines        object
internet_service      object
online_security       object
online_backup         object
device_protection     object
tech_support          object
streaming_tv          object
streaming_movies      object
contract              object
paperless_billing       bool
payment_method        object
monthly_charges      float64
total_charges        float64
churn                   bool
dtype: object

In [33]:
challenger_model

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...liblinear'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'model__C': [0.01, 0.1, ...], 'model__penalty': ['l1', 'l2'], 'model__solver': ['liblinear', 'saga']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'recall'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is dis

In [34]:
try:
    champion_uri = f"models:/churn_model@champion"
    champion_model = mlflow.sklearn.load_model(champion_uri)
    has_champion = True
except Exception:
    print("No current champion found. Challenger will be promoted automatically.")
    has_champion = False

In [35]:
champion_model

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",1.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept:

In [ ]:
from sklearn.metrics import recall_score

y_pred_champion = champion_model.predict(X_test)
recall_champion = recall_score(y_test, y_pred_champion)
print(f"champion Recall: {recall_champion:.4f}")

Challenger Recall: 0.3119


In [37]:
y_pred_challenger = challenger_model.predict(X_test)
recall_challenger = recall_score(y_test, y_pred_challenger)
print(f"Challenger Recall: {recall_challenger:.4f}")

Challenger Recall: 0.3028
